In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv("data.csv")
df_e=pd.get_dummies(df)

In [4]:
y = df.diagnosis                          # M or B 
mylist = ['Unnamed: 32','id','diagnosis']
x = df.drop(mylist,axis = 1 )
x.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
y = y.apply(lambda x: 0 if(x=='B') else 1)

In [6]:
X = x

In [7]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
neigh = KNeighborsClassifier()

In [9]:
sfs1 = sfs(neigh, k_features=14, forward=True, verbose=2, scoring='neg_mean_squared_error')

In [10]:
sfs1 = sfs1.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.8s finished

[2022-07-21 07:00:56] Features: 1/14 -- score: -0.09490762303990062[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    0.5s finished

[2022-07-21 07:00:56] Features: 2/14 -- score: -0.0791181493556901[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.5s finished

[2022-07-21 07:00:57] Features: 3/14 -- score: -0.0808104331625524[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Do

In [11]:
selected_features = list(sfs1.k_feature_names_)
print(selected_features)

['smoothness_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'smoothness_worst', 'concavity_worst', 'fractal_dimension_worst']


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [13]:
X = X[selected_features]
y = y
scaler=StandardScaler()
X=scaler.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=42)

neigh = KNeighborsClassifier(n_neighbors = 9).fit(X_train,y_train)
yhat = neigh.predict(X_test)
print("Train set Accuracy :", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy :", metrics.accuracy_score(y_test, yhat))

Train set Accuracy : 0.9501466275659824
Test set Accuracy : 0.9429824561403509


In [14]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

In [15]:
precision_score(y_test, yhat)

0.971830985915493

In [16]:
recall_score(y_test, yhat)

0.8625

In [17]:
f1_score(y_test, yhat)

0.9139072847682119

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       148
           1       0.97      0.86      0.91        80

    accuracy                           0.94       228
   macro avg       0.95      0.92      0.94       228
weighted avg       0.94      0.94      0.94       228

